# Introduction

This web scraper can extract data from the given website list. Two layers of websites were explored. 

# Step 1
## Put the mouse into the next cell, and click the left corner button of the below cell, which is the triangle button, to run the program. 

In [ ]:
from google.colab import files
uploaded = files.upload()
filename1 = next(iter(uploaded))

# Step 2
## Run the next cell until a file generated and downloaded to your local compter. 
##  ScraperResult.csv 
## is the result


In [ ]:
import io
import pandas as pd
from google.colab import files
import logging
import os
import pandas as pd
import re
from googlesearch import search
import requests
import bs4
import sys
import re
import requests
import pymongo
import datetime
from bs4 import BeautifulSoup
import multiprocessing as mp

df = pd.read_csv(io.BytesIO(uploaded[filename1]),usecols= ['Company Name','Website'])

def get_phone():
    try:
        phone = soup.select("a[href*=callto]")[0].text
        return phone
    except:
        pass

    try:
        phone = re.findall(r'\(?\b[2-9][0-9]{2}\)?[-][2-9][0-9]{2}[-][0-9]{4}\b', response.text)[0]
        return phone
    except:
        pass

    try:
        phone = re.findall(r'\(?\b[2-9][0-9]{2}\)?[-. ]?[2-9][0-9]{2}[-. ]?[0-9]{4}\b', response.text)[-1]
        return phone
    except:
        #print ('Phone number not found')
        phone = ''
        return phone


def get_email():
    try:
        email = re.findall(r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-]+\.{1}[A-Za-z]{2,3})', response.text)[-1]
        return email
    except:
        pass

    try:
        email = soup.select("a[href*=mailto]")[-1].text
        return email
    except:
        #print ('Email not found')
        email = ''
        return email

def get_address():
    try:
        str1 = soup.find(id='address').get_text(strip=True)
        str1 = str1.replace('\xa0', ' ')
        return str1
    except:
        str1 = ''
        return str1

def get_tags(str_tag):
    try:
        str1 = soup.find(str_tag).get_text(strip=True)
        str1 = str1.replace('\xa0', ' ')
        if str1=='403 Forbidden':
          str1 = ''

        return str1
    except:
        return ''

def scrape_contact_link():
    reg = re.compile(r'(?i)contact\x20{1,}us')
    url_contact =''
    for link in soup.find_all('a', href=True, text=reg): 
    # if links then goto next page
      url_contact = link['href']

    return url_contact

# Feb. 18
def scrape_contact_link1():
    reg = re.compile(r'(?i)contact\x20{1,}')
    url_contact =''
    for link in soup.find_all('a', href=True, text=reg): 
        # if links then goto next page
        url_contact = link['href']

    return url_contact

# Feb. 5, 2022
def scrape_next_layers_link(query_str):
    reg = re.compile(r'(?i)'+ query_str + '\x20{1,}us')
    url_contact =''
    for link in soup.find_all('a', href=True, text=reg): 
    # if links then goto next page
        url_contact = link['href']

    return url_contact

def scrap_contact_info_email(soup2):
    emailList=[]
    mailtos = soup.select('a[href^=mailto]')
    for i in mailtos:
        href=i['href']
        try:
            str1, str2 = href.split(':')
        except ValueError:
            break
        emailList.append(str2)

    return ' '.join(emailList)


src_df = df

for i, row in src_df.iterrows():
    url = 'http://' + row[1]
    try:
        response = requests.get(url)
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
    except:
        print ('Unsucessful: ' + str(response))
        continue

    addr = get_address()
    phone = get_phone()
    contact_email = get_email()
    title = get_tags('title')
      
    contact_email = scrap_contact_info_email(soup)
    contact_facebook = ''
    contact_twitter = ''
    contact_instagram = ''
    contact_linkedin = ''
    for link in soup.find_all(href=re.compile(r"facebook", re.I)):
        #'(?i)facebook(?i)')): 
        contact_facebook = link['href']

    for link in soup.find_all(href=re.compile(r"twitter", re.I)): 
        contact_twitter = link['href']

    for link in soup.find_all(href=re.compile(r"instagram", re.I)): 
        contact_instagram = link['href']

    for link in soup.find_all(href=re.compile(r"linkedin", re.I)): 
        contact_linkedin = link['href']


    #url2 = scrape_contact_link()
    # scrape next layers lins 
    # contact us
    query_str = 'contact'
    url2 = scrape_next_layers_link(query_str)
    if url2:
        try:
            response2 = requests.get(url2)
            soup2 = bs4.BeautifulSoup(response2.text, 'html.parser')
            # Contac Us
            contact_email = scrap_contact_info_email(soup2)
            for link in soup2.find_all(href=re.compile(r"facebook", re.I)):
                contact_facebook = link['href']

            for link in soup2.find_all(href=re.compile(r"twitter", re.I)): 
                contact_twitter = link['href']

            for link in soup2.find_all(href=re.compile(r"instagram", re.I)):
                contact_instagram = link['href']

            for link in soup2.find_all(href=re.compile(r"linkedin", re.I)): 
                contact_linkedin = link['href']

            # Mission
        except:
            print ('Unsucessful_URL2: ' + str(response2))
        #continue
    
    # About us
    query_str = 'about'
    url2 = scrape_next_layers_link(query_str)
    if url2:
        try:
            response2 = requests.get(url2)
            soup2 = bs4.BeautifulSoup(response2.text, 'html.parser')
            # About Us
            contact_email = scrap_contact_info_email(soup2)
            for link in soup2.find_all(href=re.compile(r"facebook", re.I)):
                contact_facebook = link['href']

            for link in soup2.find_all(href=re.compile(r"twitter", re.I)): 
                contact_twitter = link['href']

            for link in soup2.find_all(href=re.compile(r"instagram", re.I)):
                contact_instagram = link['href']

            for link in soup2.find_all(href=re.compile(r"linkedin", re.I)): 
                contact_linkedin = link['href'] 
        except:
            print ('Unsucessful_URL2: ' + str(response2))
        
    
    src_df.loc[i,'Phone Number'] = phone
    src_df.loc[i,'Email'] = contact_email
    src_df.loc[i,'Title'] = title
    src_df.loc[i,'Address'] = addr
    src_df.loc[i,'Linkedin'] = contact_linkedin
    src_df.loc[i,'Twitter'] = contact_twitter
    src_df.loc[i,'Instagram'] = contact_instagram
    src_df.loc[i,'Facebook'] = contact_facebook

    
src_df.to_csv('ScraperResult.csv', index=False)
files.download("ScraperResult.csv")